## Jupyter Notebook Introduction

Jupyter Notebooks allow us to run snippets of Python Code in the browser.

In [1]:
# This is a comment in a Cell

In [2]:
string = 'Hello World!'
print(string)

Hello World!


We can also have cells with text in them like this one. This is called a Markdown cell. We can **bold**, *italicize*, create equations $2x_1 + 3x_2 = 10$, and more.

## Import Python Packages

In [ ]:
import numpy as np # Array processing for numbers, strings, records, and objects
import pandas as pd # High-performance, easy-to-use data structures and data analysis tools
from concorde.tsp import TSPSolver # Python wrapper for the Concorde TSP solver
import matplotlib.pyplot as plt # Publication quality figures in python

## Reading and Manipulating Dataframes

In [ ]:
data = pd.read_csv('hygdata_v3.csv').set_index('id') # read a CSV file
data.head()

In [ ]:
# clean the dataframe to only include fields I am interested in
data = data[~data['proper'].isna()][['proper', 'dist', 'ra', 'dec', 'pmra', 'pmdec']].dropna()
data.head()

In [ ]:
# unit conversions
data['dist'] = data['dist'].apply(lambda x: x*3.262) # convert from parsecs to lightyears
data['ra'] = data['ra'].apply(lambda x: x*15) # convert from hours to degrees 
data['pmra'] = data['pmra'].apply(lambda x: x*2.7777776630942*10**(-7)) # convert to degrees per year
data['pmdec'] = data['pmdec'].apply(lambda x: x*2.7777776630942*10**(-7)) # convert to degrees per year

# rename fields
data = data.rename(columns={'ra' : 'ra_2000', 'dec' : 'dec_2000'})

# calcualte 2020 right ascension and declination
data['ra_2020'] = data['ra_2000'] + 20*data['pmra']
data['dec_2020'] = data['dec_2000'] + 20*data['pmdec']

## Visualizations

In [ ]:
# generate a visualization of the star locations
plt.figure(figsize=(20,10))
ax = plt.axes()
ax.set_facecolor('black')
plt.scatter(data.ra_2020,data.dec_2020, marker='*', color='white', s=70)
for index, row in data.iterrows():
    plt.annotate(row['proper'], 
                 (row['ra_2020'],row['dec_2020']),
                 textcoords="offset points", 
                 xytext=(5,10),
                 color= 'white',
                 ha='center')
plt.title('The Locations of 146 Stars', fontsize= 20)
plt.xlabel('Right Ascension (Degrees)', fontsize= 15)
plt.ylabel('Declination (Degrees)', fontsize= 15)
plt.show()

## Solving the TSP Instance

In [ ]:
#list(data.ra_2020)
#list(data.dec_2020)

In [ ]:
solver = TSPSolver.from_data(list(data.ra_2020), list(data.dec_2020), norm='GEO')
solver.solve()
#tour = solver.solve().tour

In [ ]:
# Generate scatter plot of star locations
plt.figure(figsize=(20,10))
ax = plt.axes()
ax.set_facecolor('black')
plt.scatter(data.ra_2020,data.dec_2020, marker='*', color='white', s=70)
for index, row in data.iterrows():
    plt.annotate(row['proper'], 
                 (row['ra_2020'],row['dec_2020']),
                 textcoords="offset points", 
                 xytext=(5,10),
                 color= 'white',
                 ha='center')
    
# Add path through these stars
tour_proper = [data.iloc[i].proper for i in tour]
tour_proper.append(tour_proper[0])
tour_ra = [data[data.proper == proper].ra_2020.to_list()[0] for proper in tour_proper]
tour_dec = [data[data.proper == proper].dec_2020.to_list()[0] for proper in tour_proper]
plt.plot(tour_ra, tour_dec, color='white')

# Set axes titles and display
plt.title('An Optimal Tour of 146 Stars', fontsize= 20)
plt.xlabel('Right Ascension (Degrees)', fontsize= 15)
plt.ylabel('Declination (Degrees)', fontsize= 15)
plt.show()